### Kudos (massive respect for sharing, even when it's not perfect)
https://www.kaggle.com/a763337092/blending-tensorflow-and-pytorch

### Pytorch source code:
https://www.kaggle.com/a763337092/neural-network-starter-pytorch-version<br/>
https://www.kaggle.com/a763337092/pytorch-resnet-starter-training

### Tensorflow source code:
https://www.kaggle.com/code1110/jane-street-with-keras-nn-overfit <br/>

P.S.<br/>
Be aware that the model maybe **overfitting**.<br/>
As you can this, this notebook it based on the effort of many other competitors, and I was so curious about their result, so I decided to experiment with their work myself ans hsare it with you in this humble notebook of mine, and hopeful to get even a slightly better Result =)


# Libs & Config

In [1]:
import os
import time
import pickle
import random
from tqdm import tqdm

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# from sklearn.metrics import log_loss, roc_auc_score this is way to track down if the model is overfitting

# Data Save/Load

In [2]:
#DATA_PATH = '../input/jane-street-market-prediction/' no need to import the data because we're going to only load pytorch and tensorflow models.
CACHE_PATH = '../input/mlp012003weights'

def save_pickle(dic, save_path):
    with open(save_path, 'wb') as f:
    # with gzip.open(save_path, 'wb') as f:
        pickle.dump(dic, f)

def load_pickle(load_path):
    with open(load_path, 'rb') as f:
    # with gzip.open(load_path, 'rb') as f:
        message_dict = pickle.load(f)
    return message_dict

f_mean = np.load(f'{CACHE_PATH}/f_mean_online.npy')

### Feature Engineering

In [3]:
# list of the features
feat_cols = [f'feature_{i}' for i in range(130)]

# list of all the features
all_feat_cols = [col for col in feat_cols]

# add two more features to the feature list
all_feat_cols.extend(['cross_41_42_43', 'cross_1_2'])

# resp 1,2,3,4
target_cols = ['action', 'action_1', 'action_2', 'action_3', 'action_4']

# PyTorch
I have used this course before, to be able to understand bacis about about Pytorch:<br/>
https://www.udacity.com/course/deep-learning-pytorch--ud188

### PyTorch Libs

In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

### PyTorch Model

I have used this course before, to be able to understand bacis about about Pytorch:<br/>
https://www.udacity.com/course/deep-learning-pytorch--ud188

In [5]:
##### Model&Data fnc
class Model(nn.Module):
    
    def __init__(self):
        
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(all_feat_cols))
        
        # Applies Dropout to the input. The Dropout layer randomly sets input units to 0 with a frequency of rate at each step during training time, 
        # which helps prevent overfitting. Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over all inputs is unchanged.
        # https://keras.io/api/layers/regularization_layers/dropout/
        
        # Dropout is a regularization technique. You should use it only to reduce variance (validation performance vs training performance).
        # It is not intended to reduce the bias, and you should not use it in this way.
        # Dropout works by probabilistically removing, or “dropping out,” inputs to a layer, 
        # which may be input variables in the data sample or activations from a previous layer.
        # With dropout (dropout rate less than some small value), the accuracy will gradually increase and loss will gradually decrease first
        # When you increase dropout beyond a certain threshold, it results in the model not being able to fit properly.
        # https://stackoverflow.com/questions/59044351/can-dropout-increases-training-data-performance
        
        # Dropout prevents overfitting due to a layer's "over-reliance" on a few of its inputs. Because these inputs aren't always present 
        # during training (i.e. they are dropped at random), the layer learns to use all of its inputs, improving generalization.
        # https://stats.stackexchange.com/questions/374742/does-dropout-regularization-prevent-overfitting-due-to-too-many-iterations
        
        # The default interpretation of the dropout hyperparameter is the probability of training a given node in a layer, 
        # where 1.0 means no dropout, and 0.0 means no outputs from the layer. 
        # A good value for dropout in a hidden layer is between 0.5 and 0.8. Input layers use a larger dropout rate, such as of 0.8.
        # https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/
        
        self.dropout0 = nn.Dropout(0.8) # 0.2

        dropout_rate = 0.5 # 0.2
        hidden_size = 256
        
        self.dense1 = nn.Linear(len(all_feat_cols), hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size+len(all_feat_cols), hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        self.dense5 = nn.Linear(hidden_size+hidden_size, len(target_cols))

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()

    def forward(self, x):
        
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x

### PyTorch on CPU or GPU 

In [6]:
# Since we're going to onyl load the model, then use it for inference, then it's better to use cpu
# Otherwise, you want to retrain the model, then enable the GPU for faster calculations

if torch.cuda.is_available():
    print('using device: cuda')
    torch.device("cuda:0")
else:
    print('using device: cpu')
    device = torch.device('cpu')

using device: cpu


### PyTorch Load Model weights

In [7]:
NFOLDS = 5

model_list = []
tmp = np.zeros(len(feat_cols))
for _fold in range(NFOLDS):
    torch.cuda.empty_cache()
    model = Model()
    model.to(device)
    model_weights = f"{CACHE_PATH}/online_model{_fold}.pth"
    #model.load_state_dict(torch.load(model_weights))
    model.load_state_dict(torch.load(model_weights, map_location=device))
    model.eval()
    model_list.append(model)

# TensorFlow
### Libs

In [8]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import tensorflow_addons as tfa

### TensorFlow model

To understand the following code, refer to my detailed notebook: <br/>
[Explanation of the model](https://www.kaggle.com/mouafekmk/simple-mlp)


In [9]:
SEED = 1111

np.random.seed(SEED)

# fit
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
    
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model

epochs = 300
batch_size = 4096
hidden_units = [160, 160, 160]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

tf.keras.backend.clear_session()
tf.random.set_seed(SEED)
clf = create_mlp(len(feat_cols), 5, hidden_units, dropout_rates, label_smoothing, learning_rate)

### TensorFlow Load model weights

In [10]:
# Fit the model and save it with 
#clf.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=2)
#clf.save(f'model.h5')

# Load the Fitted model
# !ls ../input/jane-street-with-keras-nn-overfit/
clf.load_weights('../input/jane-street-with-keras-nn-overfit/model.h5')

# If you have several models, the you can store into a list
#tf_models = [clf]



# Inference

In [11]:
th = 0.5 # 0.5 0.501 0.0502 0.053 did not have an effect
import janestreet
janestreet.competition.make_env.__called__ = False

env = janestreet.make_env()
env_iter = env.iter_test()

#test_df has one single row of data with all the feautures
# pred_dfd has 1 or 0 which is an action that comes from test_df

for (test_df, pred_df) in tqdm(env_iter):

    # Data Processing
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, feat_cols].values
        
        # if there is only one value eqauls nan, then x_tt.sum will return nan, then np.isnan will be true, and enters the loop
        if np.isnan(x_tt.sum()):
            # Replace NaN with zero and infinity with large finite numbers (default behaviour)
            x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean # the score goew down if we take the if statement

    
        cross_41_42_43 = x_tt[:, 41] + x_tt[:, 42] + x_tt[:, 43]
        cross_1_2 = x_tt[:, 1] / (x_tt[:, 2] + 1e-5)
        feature_inp = np.concatenate((x_tt, np.array(cross_41_42_43).reshape(x_tt.shape[0], 1), np.array(cross_1_2).reshape(x_tt.shape[0], 1),), axis=1)

        
        # PytTorch prediction 
        torch_pred = np.zeros((1, len(target_cols)))
        for model in model_list:
            torch_pred += model(torch.tensor(feature_inp, dtype=torch.float).to(device)).sigmoid().detach().cpu().numpy() / NFOLDS
        torch_pred = np.median(torch_pred)

        
        # TensofFlow prediction
        # I foyu have several TF models then use
        #tf_pred = np.median(np.mean([model(x_tt, training = False).numpy() for model in tf_models],axis=0))
        # If you have only one model then use
        tf_pred = np.median(clf(x_tt))

        
        # PyTorch and TensorFlow Average prediction
        pred = torch_pred * 0.4 + tf_pred * 0.6
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
        
    else:
        pred_df.action = 0
        
    env.predict(pred_df)

15219it [05:06, 49.70it/s]
